In [ ]:
!pip install mlflow

In [ ]:
import tensorflow as tf
import mlflow
import mlflow.keras

In [ ]:
# Ensure TensorFlow is using GPU
if len(tf.config.list_physical_devices('GPU')) > 0:
    print("Using GPU")
else:
    print("Using CPU")

In [ ]:
# Enable MLflow autologging
mlflow.tensorflow.autolog()

# Set the experiment name (creates the experiment if it doesn't exist)
experiment_name = "simple_tensorflow_example"
mlflow.set_experiment(experiment_name)

# Load the MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the data to [0, 1] range
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define a simple sequential model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Start an MLflow run under the created experiment
with mlflow.start_run() as run:
    # Train the model
    model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

    # Evaluate the model
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

    print(f"\nTest accuracy: {test_acc}")

    # Register the model using the Keras API
    model_name = "MNIST_TensorFlow_Model"
    
    # Log and register the Keras model in the MLflow model registry
    mlflow.keras.log_model(model, "model", registered_model_name=model_name)

    # End the MLflow run
    mlflow.end_run()

print(f"Model registered in the MLflow Model Registry as '{model_name}'")